In [1]:
import numpy as np
import tensorflow as tf
from keras.utils import np_utils

2022-06-14 09:29:35.850612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 09:29:35.850649: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import string
import re
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def remove_numbers(text_in):
  for ele in text_in.split():
    if ele.isdigit():
        text_in = text_in.replace(ele, "@")
  for character in text_in:
    if character.isdigit():
        text_in = text_in.replace(character, "@")
  return text_in


def remove_special_characters(text):
  chars = re.escape(string.punctuation)
  return re.sub(r'['+chars+']', '', text)

def preprocess(text_in):
    text = clean_text(text_in)
    text = remove_special_characters(text)
    # text = remove_numbers(text)
    text = text.lower()
    return text

text = 'Chàng trai 9X Quảng Trị khởi nghiệp từ nấm sò'
preprocess(text)

'chàng trai 9x quảng trị khởi nghiệp từ nấm sò'

# Creating character/word mappings

In [3]:
characters = []
n_to_char = {}
char_to_n = {}

with open('./data/data.txt') as f:
    data = f.readlines()

    characters = sorted(list(set(data)))
    n_to_char = {n : char for n, char in enumerate(characters)}
    char_to_n = {char : n for n, char in enumerate(characters)}
    

# Data preprocessing

In [4]:
X = []
y = []
length = len(data)
seq_length = 32
for i in range(length - seq_length):
    sequence = data[i: i + seq_length]
    print(sequence)
    label = data[i + seq_length]
    print(label)
    X.append([char_to_n[char] for char in sequence])
    y.append([char_to_n[label]])
print(X[99])
print(y[99])



['Trong này hơi tối nhỉ bật đèn trần lên\n', 'Trong này sáng quá tắt bớt đèn.\n', 'Trong này hơi lạnh nhỉ tắt máy lạnh.\n', 'Trong này hơi lạnh nhỉ tắt quạt.\n', 'Trong này hơi nóng nhỉ bật máy lạnh.\n', 'Trong này hơi nóng, bật quạt.\n', 'Không cần bật máy lạnh đâu, bật quạt được rồi.\n', 'Bật điện phòng ngủ.\n', 'Mở điện phòng ngủ\n', 'Bật điện phòng khách lầu 1.\n', 'Bật điện phòng khách tầng 1.\n', 'Bật điện phòng khách tầng 2.\n', 'Bật điện phòng khách lầu trệt.\n', 'Bật điện phòng khách lầu 3.\n', 'Bật điện phòng khách lầu 4.\n', 'Mở điện phòng khách lầu 1.\n', 'Mở điện phòng khách lầu 2.\n', 'Mở điện phòng khách lầu 3.\n', 'Mở điện phòng khách lầu 4.\n', 'Mở điện phòng khách lầu 5.\n', 'Mở điện phòng khách tầng 1.\n', 'Mở điện phòng khách tầng 2.\n', 'Mở điện phòng khách tầng 3.\n', 'Mở điện phòng khách tầng 4.\n', 'Mở điện phòng khách tầng 5.\n', 'Bật điện phòng ngủ hiện tại.\n', 'Bật điện phòng ngủ lầu 1.\n', 'Bật điện phòng ngủ lầu 2.\n', 'Bật điện phòng ngủ lầu 3.\n', 'Bật đ

In [5]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
y_modified = np_utils.to_categorical(y)

# Modeling

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Flatten, Embedding, BatchNormalization, LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(1024, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


2022-06-14 09:29:42.264768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-14 09:29:42.266657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-14 09:29:42.267148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 09:29:42.267231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-14 09:29:42.267281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not lo

In [7]:
from tensorflow import keras
import tensorflow as tf
from datetime import datetime

dt = datetime.now()
dt_s = '_'.join(str(dt).split(' '))
print(dt_s)


checkpoint_path = f'./model/generate_text_by_char/test_lstm_{dt_s}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    monitor='loss',
    save_weights_only=True,
    mode='min',
    save_best_only=True),

    keras.callbacks.EarlyStopping(
    verbose=1,
    monitor='loss',
    patience=25)
]
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

2022-06-14_09:29:53.472795


In [8]:
history = model.fit(X_modified, y_modified, epochs=1000, verbose=1, callbacks=callbacks, batch_size=64)


Epoch 1/1000
6/6 [==============================] - ETA: 0s - loss: 5.9181 - accuracy: 0.0000e+00
Epoch 1: loss improved from inf to 5.91814, saving model to ./model/generate_text_by_char/test_lstm_2022-06-14_09:29:53.472795.ckpt
6/6 [==============================] - 32s 3s/step - loss: 5.9181 - accuracy: 0.0000e+00
Epoch 2/1000
6/6 [==============================] - ETA: 0s - loss: 5.9347 - accuracy: 0.0028
Epoch 2: loss did not improve from 5.91814
6/6 [==============================] - 19s 3s/step - loss: 5.9347 - accuracy: 0.0028
Epoch 3/1000
6/6 [==============================] - ETA: 0s - loss: 5.9349 - accuracy: 0.0056
Epoch 3: loss did not improve from 5.91814
6/6 [==============================] - 18s 3s/step - loss: 5.9349 - accuracy: 0.0056
Epoch 4/1000
6/6 [==============================] - ETA: 0s - loss: 5.9130 - accuracy: 0.0056
Epoch 4: loss improved from 5.91814 to 5.91299, saving model to ./model/generate_text_by_char/test_lstm_2022-06-14_09:29:53.472795.ckpt
6/6 [==

# Generating Text

In [12]:
model.load_weights(checkpoint_path)

In [22]:
string_mapped = X[200]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(4):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    # print(f'x: {x} - x shape: {x.shape}')
    x = x / float(len(characters))
    # print(f'x: {x} - x shape: {x.shape}')

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]



In [23]:
#combining text
txt = ""
for char in full_string:
    txt += char
txt

'Tắt đèn sau một giờ.\nBật đèn.\nNhấp nháy đèn cho tôi.\nBật tủ lạnh.\nBật lò nướng.\nBật hệ thống âm nhạc.\nTắt đèn trên gác xép.\nTắt đèn ở tầng hầm.\nTắt đèn trong bếp.\nTắt đèn trong thư viện.\nTắt đèn trong bếp.\nTắt đèn trong phòng ăn.\nTắt đèn trong hầm.\nTắt đèn trên gác xép.\nTắt đèn ở tầng hầm.\nTắt đèn trong bếp.\nTắt đèn trong thư viện.\nTắt đèn trong bếp.\nTắt đèn trong phòng ăn.\nTắt đèn trong hầm.\nTắt thứ gì đó sau 45 phút.\nBật một cái gì đó.\nĐặt một thiết bị ngẫu nhiên trong 7 ngày.\nđèn chiếu sáng.\nBạn có thể làm cho nó bớt tối trong phòng tắm vào ngày mai?\nBạn có thể làm cho nó bớt tối trong nhà vệ sinh vào ngày mai?\nBật đèn trên gác mái sau một giờ nữa.\nBạn có thể tắt đèn trong hầm được không?\nTôi sẽ phát điên lên nếu bạn bật đèn ngay bây giờ.\nTrời tối trong hầm.\nNó tối trong phòng tắm.\nNó là tối trong phòng khách.\nTrong thư viện trời tối.\nĐó là tối trong nhà vệ sinh.\nTrời tối trên gác xép.\nTrời tối trong tầng hầm.\n'